# PPO vs GRPO on MountainCarContinuous-v0

## Introduction

This notebook compares two reinforcement learning algorithms on the **MountainCarContinuous-v0** environment:

- **PPO (Proximal Policy Optimization)**: A popular policy gradient method from `stable-baselines3` that uses a value function (critic) and clipped policy gradients for stable training.

- **GRPO (Group Relative Policy Optimization)**: An algorithm implemented in this repository (`sb3_contrib`) that uses relative performance within a group instead of a learned value function. GRPO normalizes advantages within groups of samples, providing stable gradient estimates without the need for a critic network.

### Environment: MountainCarContinuous-v0
The goal is to drive an underpowered car up a steep hill. The car must build momentum by rocking back and forth. The state space includes position and velocity, and the action is a continuous force applied to the car.

### Outcome
By the end of this notebook, you will see:
1. Training curves comparing episodic rewards for PPO and GRPO
2. A side-by-side video of both trained agents for direct visual comparison

## 1) Setup and Imports

In [1]:
import os
import gymnasium as gym
import numpy as np
import matplotlib.pyplot as plt

# stable-baselines3 imports
from stable_baselines3 import PPO
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.callbacks import BaseCallback

# sb3_contrib imports (GRPO from this repository)
from sb3_contrib import GRPO

# TensorBoard logging
from torch.utils.tensorboard import SummaryWriter

# Video recording
from gymnasium.wrappers import RecordVideo

# Video composition (moviepy 2.x uses moviepy.video.io instead of moviepy.editor)
try:
    from moviepy.editor import VideoFileClip, clips_array
except ImportError:
    # For moviepy 2.x
    from moviepy import VideoFileClip, concatenate_videoclips

# For displaying video in notebook
from IPython.display import HTML


2025-12-05 14:21:22.010669: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-12-05 14:21:22.049756: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-12-05 14:21:22.987992: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-12-05 14:21:22.987992: I tensorflow/core/util/port.cc:153] oneDNN custom operations ar

In [2]:
# Print versions for reproducibility
import stable_baselines3
import sb3_contrib

print(f"gymnasium version: {gym.__version__}")
print(f"stable-baselines3 version: {stable_baselines3.__version__}")
print(f"sb3_contrib version: {sb3_contrib.__version__}")

gymnasium version: 1.2.2
stable-baselines3 version: 2.7.0
sb3_contrib version: 2.7.1a3


## 2) Environment Helpers

In [3]:
from typing import List, Optional


def make_env(seed: Optional[int] = None) -> gym.Env:
    """
    Create a MountainCarContinuous-v0 environment.
    
    Args:
        seed: Optional random seed for reproducibility.
        
    Returns:
        The created gymnasium environment.
    """
    env = gym.make("MountainCarContinuous-v0")
    if seed is not None:
        env.reset(seed=seed)
    return env

## 3) Training Configuration

In [4]:
# Training configuration
# MountainCarContinuous-v0 requires extensive training
TOTAL_TIMESTEPS = 2_000_000  # Increased to 2M - this environment is very difficult
SEED = 0                      # Random seed for reproducibility
N_EVAL_EPISODES = 10          # Number of episodes for final evaluation
VIDEO_EPISODES = 1            # Number of episodes to record for video

# Hyperparameters from RL Baselines3 Zoo (proven to work)
# Source: https://github.com/DLR-RM/rl-baselines3-zoo/blob/master/hyperparams/ppo.yml
PPO_PARAMS = {
    'learning_rate': 7e-4,      # Lower LR for more stable learning (was 1e-3)
    'n_steps': 8,               # Much smaller for faster value updates (was 16)
    'batch_size': 256,          # Larger batches for stable gradients (was 16)
    'n_epochs': 10,             # Keep default
    'gamma': 0.9999,            # Very high discount - long-term planning crucial
    'gae_lambda': 0.94,         # Lower GAE for less bias (was 0.98)
    'clip_range': 0.2,          # Default clip range
    'ent_coef': 0.0,            # No entropy bonus
    'vf_coef': 0.5,             # Value function coefficient
    'max_grad_norm': 0.5,       # Gradient clipping
}

GRPO_PARAMS = {
    'learning_rate': 7e-4,      # Match PPO (was 1e-3)
    'n_steps': 8,               # Match PPO (was 16)
    'batch_size': 256,          # Match PPO (was 16)
    'gamma': 0.9999,            # Match PPO
    'group_size': 32,           # Larger groups for better statistics (was 8)
    'kl_coef': 0.05,            # Stronger KL penalty (was 0.02)
    'clip_range': 0.2,          # Match PPO
    'max_grad_norm': 0.5,       # Gradient clipping
}

# Ensure output directories exist
os.makedirs('examples/models', exist_ok=True)
os.makedirs('examples/videos', exist_ok=True)
os.makedirs('/tmp/tensorboard_logs', exist_ok=True)

print(f'Training for {TOTAL_TIMESTEPS:,} timesteps (~{TOTAL_TIMESTEPS // 200} episodes)')
print(f'PPO params: lr={PPO_PARAMS["learning_rate"]}, n_steps={PPO_PARAMS["n_steps"]}, batch={PPO_PARAMS["batch_size"]}, gamma={PPO_PARAMS["gamma"]}')
print(f'GRPO params: lr={GRPO_PARAMS["learning_rate"]}, n_steps={GRPO_PARAMS["n_steps"]}, batch={GRPO_PARAMS["batch_size"]}, gamma={GRPO_PARAMS["gamma"]}')

Training for 2,000,000 timesteps (~10000 episodes)
PPO params: lr=0.0007, n_steps=8, batch=256, gamma=0.9999
GRPO params: lr=0.0007, n_steps=8, batch=256, gamma=0.9999


## 4) Callback for Logging Training Rewards

We create a simple callback to track episodic rewards during training.

In [5]:
class RewardLoggerCallback(BaseCallback):
    """
    Callback for logging episode rewards during training.
    """
    
    def __init__(self, verbose: int = 0):
        super().__init__(verbose)
        self.episode_rewards: List[float] = []
        self.episode_lengths: List[int] = []
        self.timesteps: List[int] = []
        
    def _on_step(self) -> bool:
        # Check if any episode has finished
        if self.locals.get("infos"):
            for info in self.locals["infos"]:
                if "episode" in info:
                    self.episode_rewards.append(info["episode"]["r"])
                    self.episode_lengths.append(info["episode"]["l"])
                    self.timesteps.append(self.num_timesteps)
        return True

## 5) Training PPO Baseline

**PPO (Proximal Policy Optimization)** is a popular on-policy algorithm that:
- Uses a value function (critic) to estimate expected returns
- Uses clipped policy gradients to prevent large policy updates
- Is known for stable training across many environments

In [6]:
# Create environment with Monitor wrapper to track episode stats
ppo_env = Monitor(make_env(SEED))

# Create PPO model with tuned hyperparameters
ppo_model = PPO(
    'MlpPolicy',
    ppo_env,
    seed=SEED,
    verbose=0,
    tensorboard_log='/tmp/tensorboard_logs',
    **PPO_PARAMS,  # Apply tuned hyperparameters
)

print(f'PPO config: lr={ppo_model.learning_rate}, n_steps={ppo_model.n_steps}, batch={ppo_model.batch_size}, gamma={ppo_model.gamma}')

# Create callback for logging rewards
ppo_callback = RewardLoggerCallback()

# Train PPO
print('\nTraining PPO...')
ppo_model.learn(
    total_timesteps=TOTAL_TIMESTEPS,
    callback=ppo_callback,
    progress_bar=True,
    tb_log_name='PPO',  # This creates PPO_1, PPO_2, etc.
)

# Save the trained model
ppo_model.save('examples/models/ppo_mountaincar')
print('PPO model saved to examples/models/ppo_mountaincar.zip')

/home/rainer/anaconda3/envs/rlracing/lib/python3.11/site-packages/stable_baselines3/ppo/ppo.py:155: UserWarning: You have specified a mini-batch size of 256, but because the `RolloutBuffer` is of size `n_steps * n_envs = 8`, after every 0 untruncated mini-batches, there will be a truncated mini-batch of size 8
We recommend using a `batch_size` that is a factor of `n_steps * n_envs`.
Info: (n_steps=8 and n_envs=1)
  warnings.warn(


Output()

/home/rainer/anaconda3/envs/rlracing/lib/python3.11/site-packages/stable_baselines3/common/on_policy_algorithm.py:150: UserWarning: You are trying to run PPO on the GPU, but it is primarily intended to run on the CPU when not using a CNN policy (you are using ActorCriticPolicy which should be a MlpPolicy). See https://github.com/DLR-RM/stable-baselines3/issues/1245 for more info. You can pass `device='cpu'` or `export CUDA_VISIBLE_DEVICES=` to force using the CPU.Note: The model will train, but the GPU utilization will be poor and the training might take longer than on CPU.
  warnings.warn(


PPO config: lr=0.0007, n_steps=8, batch=256, gamma=0.9999

Training PPO...


PPO model saved to examples/models/ppo_mountaincar.zip


In [7]:
# Evaluate PPO
ppo_eval_env = make_env(SEED + 100)  # Different seed for evaluation
ppo_mean_reward, ppo_std_reward = evaluate_policy(
    ppo_model, ppo_eval_env, n_eval_episodes=N_EVAL_EPISODES, deterministic=True
)
print(f"PPO Evaluation: Mean reward = {ppo_mean_reward:.2f} +/- {ppo_std_reward:.2f}")
ppo_eval_env.close()

# Store training rewards for plotting
ppo_rewards = np.array(ppo_callback.episode_rewards)
ppo_timesteps = np.array(ppo_callback.timesteps)

/home/rainer/anaconda3/envs/rlracing/lib/python3.11/site-packages/stable_baselines3/common/evaluation.py:70: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


PPO Evaluation: Mean reward = -99.90 +/- 0.00


## 6) Training GRPO

**GRPO (Group Relative Policy Optimization)** is an algorithm that:
- Uses relative performance within a group instead of a learned value function
- Normalizes advantages within groups of samples for stable gradient estimates
- Reduces computational overhead by eliminating the critic network
- Uses KL divergence regularization to prevent policy collapse

In [8]:
# Create environment with Monitor wrapper to track episode stats
grpo_env = Monitor(make_env(SEED))

# Create GRPO model with tuned hyperparameters
grpo_model = GRPO(
    'MlpPolicy',
    grpo_env,
    seed=SEED,
    verbose=0,
    tensorboard_log='/tmp/tensorboard_logs',
    **GRPO_PARAMS,  # Apply tuned hyperparameters
)

print(f'GRPO config: lr={grpo_model.learning_rate}, n_steps={grpo_model.n_steps}, batch={grpo_model.batch_size}, gamma={grpo_model.gamma}')

# Create callback for logging rewards
grpo_callback = RewardLoggerCallback()

# Train GRPO
print('\nTraining GRPO...')
grpo_model.learn(
    total_timesteps=TOTAL_TIMESTEPS,
    callback=grpo_callback,
    progress_bar=True,
    tb_log_name='GRPO',  # This creates GRPO_1, GRPO_2, etc.
)

# Save the trained model
grpo_model.save('examples/models/grpo_mountaincar')
print('GRPO model saved to examples/models/grpo_mountaincar.zip')

Output()

GRPO config: lr=0.0007, n_steps=8, batch=256, gamma=0.9999

Training GRPO...


/home/rainer/pr/github/stable-baselines3-contrib/sb3_contrib/grpo/grpo.py:161: UserWarning: You have specified a mini-batch size of 256, but because the `RolloutBuffer` is of size `n_steps * n_envs = 8`, after every 0 untruncated mini-batches, there will be a truncated mini-batch of size 8
We recommend using a `batch_size` that is a factor of `n_steps * n_envs`.
Info: (n_steps=8 and n_envs=1)
  warnings.warn(
/home/rainer/anaconda3/envs/rlracing/lib/python3.11/site-packages/stable_baselines3/common/on_policy_algorithm.py:150: UserWarning: You are trying to run GRPO on the GPU, but it is primarily intended to run on the CPU when not using a CNN policy (you are using ActorCriticPolicy which should be a MlpPolicy). See https://github.com/DLR-RM/stable-baselines3/issues/1245 for more info. You can pass `device='cpu'` or `export CUDA_VISIBLE_DEVICES=` to force using the CPU.Note: The model will train, but the GPU utilization will be poor and the training might take longer than on CPU.
  war

KeyboardInterrupt: 

In [ ]:
# Evaluate GRPO
grpo_eval_env = make_env(SEED + 100)  # Different seed for evaluation
grpo_mean_reward, grpo_std_reward = evaluate_policy(
    grpo_model, grpo_eval_env, n_eval_episodes=N_EVAL_EPISODES, deterministic=True
)
print(f"GRPO Evaluation: Mean reward = {grpo_mean_reward:.2f} +/- {grpo_std_reward:.2f}")
grpo_eval_env.close()

# Store training rewards for plotting
grpo_rewards = np.array(grpo_callback.episode_rewards)
grpo_timesteps = np.array(grpo_callback.timesteps)

## 7) Training Curve Comparison

In [ ]:
def smooth_rewards(rewards: np.ndarray, window: int = 10) -> np.ndarray:
    """
    Apply a simple moving average to smooth reward curves.
    
    Args:
        rewards: Array of episode rewards.
        window: Window size for moving average.
        
    Returns:
        Smoothed rewards array.
    """
    if len(rewards) < window:
        return rewards
    return np.convolve(rewards, np.ones(window) / window, mode="valid")


# Create figure for comparison
fig, ax = plt.subplots(figsize=(12, 6))

# Plot PPO training curve
if len(ppo_rewards) > 0:
    ppo_smoothed = smooth_rewards(ppo_rewards)
    ax.plot(
        range(len(ppo_smoothed)),
        ppo_smoothed,
        label="PPO",
        color="blue",
        alpha=0.8,
    )
    # Plot raw rewards with transparency
    ax.plot(
        range(len(ppo_rewards)),
        ppo_rewards,
        color="blue",
        alpha=0.2,
    )

# Plot GRPO training curve
if len(grpo_rewards) > 0:
    grpo_smoothed = smooth_rewards(grpo_rewards)
    ax.plot(
        range(len(grpo_smoothed)),
        grpo_smoothed,
        label="GRPO",
        color="red",
        alpha=0.8,
    )
    # Plot raw rewards with transparency
    ax.plot(
        range(len(grpo_rewards)),
        grpo_rewards,
        color="red",
        alpha=0.2,
    )

ax.set_xlabel("Episode")
ax.set_ylabel("Episodic Reward")
ax.set_title("PPO vs GRPO on MountainCarContinuous-v0")
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 8) Video Recording and Side-by-Side Visualization

Now we record videos of both trained agents and create a side-by-side comparison.

In [ ]:
def record_video(model, model_name: str, video_dir: str) -> str:
    """
    Record a video of the trained agent.
    
    Args:
        model: The trained RL model.
        model_name: Name of the model (for file naming).
        video_dir: Directory to save the video.
        
    Returns:
        Path to the recorded video file.
    """
    # Create video directory
    video_folder = os.path.join(video_dir, model_name)
    os.makedirs(video_folder, exist_ok=True)
    
    # Create environment with video recorder
    env = gym.make("MountainCarContinuous-v0", render_mode="rgb_array")
    env = RecordVideo(
        env,
        video_folder=video_folder,
        episode_trigger=lambda x: True,  # Record all episodes
        name_prefix=model_name,
    )
    
    # Run episodes
    for episode in range(VIDEO_EPISODES):
        obs, info = env.reset()
        done = False
        while not done:
            action, _ = model.predict(obs, deterministic=True)
            obs, reward, terminated, truncated, info = env.step(action)
            done = terminated or truncated
    
    env.close()
    
    # Find the generated video file
    video_files = [f for f in os.listdir(video_folder) if f.endswith(".mp4")]
    if video_files:
        return os.path.join(video_folder, video_files[0])
    return ""

In [ ]:
# Record videos for both models
print("Recording PPO video...")
ppo_video_path = record_video(ppo_model, "ppo_mountaincar", "examples/videos")
print(f"PPO video saved to: {ppo_video_path}")

print("\nRecording GRPO video...")
grpo_video_path = record_video(grpo_model, "grpo_mountaincar", "examples/videos")
print(f"GRPO video saved to: {grpo_video_path}")

## 9) Side-by-Side Video Composition

Using moviepy, we create a side-by-side video comparison:
- **Left**: PPO agent
- **Right**: GRPO agent

This allows for direct visual comparison of how each algorithm solves the MountainCar task.

In [ ]:
# Load the video clips
ppo_clip = VideoFileClip(ppo_video_path)
grpo_clip = VideoFileClip(grpo_video_path)

# Resize to same height if needed (moviepy 2.x uses resized method)
target_height = min(ppo_clip.h, grpo_clip.h)
try:
    # moviepy 1.x
    ppo_clip_resized = ppo_clip.resize(height=target_height)
    grpo_clip_resized = grpo_clip.resize(height=target_height)
except AttributeError:
    # moviepy 2.x
    scale_ppo = target_height / ppo_clip.h
    scale_grpo = target_height / grpo_clip.h
    ppo_clip_resized = ppo_clip.resized(scale_ppo)
    grpo_clip_resized = grpo_clip.resized(scale_grpo)

# Make clips the same duration (use the shorter one)
min_duration = min(ppo_clip_resized.duration, grpo_clip_resized.duration)
try:
    # moviepy 1.x
    ppo_clip_resized = ppo_clip_resized.subclip(0, min_duration)
    grpo_clip_resized = grpo_clip_resized.subclip(0, min_duration)
except AttributeError:
    # moviepy 2.x
    ppo_clip_resized = ppo_clip_resized.subclipped(0, min_duration)
    grpo_clip_resized = grpo_clip_resized.subclipped(0, min_duration)

# Create side-by-side composition
try:
    # Try moviepy 1.x style
    from moviepy.editor import clips_array
    side_by_side = clips_array([[ppo_clip_resized, grpo_clip_resized]])
except (ImportError, AttributeError):
    # For moviepy 2.x - create side by side manually
    from moviepy import CompositeVideoClip
    
    # Calculate total width
    total_width = ppo_clip_resized.w + grpo_clip_resized.w
    
    # Position clips side by side
    side_by_side = CompositeVideoClip(
        [
            ppo_clip_resized.with_position((0, 0)),
            grpo_clip_resized.with_position((ppo_clip_resized.w, 0))
        ],
        size=(total_width, target_height)
    )

# Save the composite video
output_path = "examples/videos/ppo_vs_grpo_mountaincar_side_by_side.mp4"
side_by_side.write_videofile(output_path, fps=30, codec="libx264")

# Close clips
ppo_clip.close()
grpo_clip.close()

print(f"\nSide-by-side video saved to: {output_path}")


## 10) Display Side-by-Side Video

The video below shows:
- **Left**: PPO agent solving MountainCarContinuous-v0
- **Right**: GRPO agent solving MountainCarContinuous-v0

In [ ]:
# Display the side-by-side video in the notebook
video_html = f"""
<video width="800" controls>
    <source src="{output_path}" type="video/mp4">
    Your browser does not support the video tag.
</video>
"""

HTML(video_html)

## 11) View Results in TensorBoard

TensorBoard logs have been saved during training in `/tmp/tensorboard_logs/`.

### Option 1: Launch TensorBoard from terminal
```bash
tensorboard --logdir /tmp/tensorboard_logs
```
Then open your browser to http://localhost:6006

### Option 2: Launch TensorBoard in Jupyter (next cell)

### Run Organization:
- **PPO_1, PPO_2, ...**: PPO training runs
- **GRPO_1, GRPO_2, ...**: GRPO training runs

Each time you train, a new numbered run is created (e.g., PPO_1, PPO_2) so you can compare multiple training runs.

### Metrics Available:
- **rollout/ep_rew_mean**: Mean episode reward (should reach 90-100 for success)
- **rollout/ep_len_mean**: Mean episode length (should decrease from ~1000 to ~100-300)
- **train/learning_rate**: Learning rate over time
- **train/loss**: Policy and value losses
- **train/policy_gradient_loss**: Policy gradient loss
- **train/value_loss**: Value function loss (PPO only)

### Comparing PPO vs GRPO:
In TensorBoard, you can overlay the PPO and GRPO runs to directly compare:
- Training efficiency (how quickly rewards improve)
- Final performance (maximum achieved reward)
- Stability (variance in rewards over time)

**Tip**: Use the TensorBoard UI to select which runs to display (e.g., PPO_1 vs GRPO_1) for side-by-side comparison.

In [ ]:
# Launch TensorBoard in notebook (optional)
%load_ext tensorboard
%tensorboard --logdir /tmp/tensorboard_logs

## Summary

This notebook demonstrated a comparison between PPO and GRPO on the MountainCarContinuous-v0 environment.

### Key Differences:

| Aspect | PPO | GRPO |
|--------|-----|------|
| Value Function | Uses a learned critic | No critic needed |
| Advantage Estimation | GAE with value function | Group-relative normalization |
| Computational Cost | Higher (two networks) | Lower (single network) |
| Regularization | Clipped policy gradients | KL divergence + clipping |

### Files Generated:
- `examples/models/ppo_mountaincar.zip` - Trained PPO model
- `examples/models/grpo_mountaincar.zip` - Trained GRPO model
- `examples/videos/ppo_mountaincar/` - PPO agent video
- `examples/videos/grpo_mountaincar/` - GRPO agent video
- `examples/videos/ppo_vs_grpo_mountaincar_side_by_side.mp4` - Side-by-side comparison

In [ ]:
# Cleanup: Close training environments
ppo_env.close()
grpo_env.close()

print("All environments closed.")

(rlracing) rainer@borg-mini:~/pr/github/stable-baselines3-contrib$ python -m rl_zoo3.train --algo ppo --env MountainCar-v0 -n 50000 -optimize --n-trials 1000 --n-jobs 2 --sampler random --pruner median
